In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.autograd import Function
from torchvision.models.vision_transformer import vit_b_16
from torchvision.models import ResNet152_Weights
from torchvision.models import ViT_B_16_Weights
!pip install peft
import torch
import os
from transformers import ViTForImageClassification
from peft import LoraConfig,LoraModel
from transformers import ViTForImageClassification


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 7.0 MB/s eta 0:00:00:00:01


In [2]:
from huggingface_hub import login
login()

In [3]:
class ModifiedViTForImageClassification(ViTForImageClassification):
    def forward(self, pixel_values, **kwargs):
        outputs = self.vit(pixel_values, **kwargs)
        return outputs[0][:,0]  # Returns class token as output
    
# Define the Gradient Reversal Layer (GRL)
class GradientReversalLayer(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha
        return output, None

# Contrastive Loss (Cosine Similarity)
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin
    
    def forward(self, features_a, features_b, label):
        cosine_sim = F.cosine_similarity(features_a, features_b)
        loss = (1 - label) * 0.5 * torch.pow(cosine_sim, 2) + \
               label * 0.5 * torch.pow(torch.clamp(self.margin - cosine_sim, min=0.0), 2)
        return loss.mean()

# Custom model with pretrained ResNet and ViT with GRL
class CustomResNetPretrainedViTDANN(nn.Module):
    def __init__(self, num_classes=18, num_domains=4, vit_dim=768, dropout_p=0.5):  # ViT base has output dim of 768
        super(CustomResNetPretrainedViTDANN, self).__init__()
        
        # Load the pretrained ResNet
        self.resnet = models.resnet152(weights=ResNet152_Weights.DEFAULT)
        for param in self.resnet.parameters():
            param.requires_grad = False  # Freeze ResNet parameters
            
        # Keep the last layer trainable
        for param in self.resnet.layer4.parameters():
            param.requires_grad = True
            
        for param in self.resnet.layer3.parameters():
            param.requires_grad = True
        
        self.resnet.fc = nn.Identity()  # Remove the classification head to output features

        self.resnet_to_vit_projection = nn.Linear(2048, vit_dim)

        # Instance Normalization for ResNet features
        self.instance_norm_resnet = nn.InstanceNorm1d(vit_dim)

        # Load pretrained Vision Transformer (ViT)
#         self.vit = vit_b_16(weights=ViT_B_16_Weights.DEFAULT)
#         self.vit_to_projection = nn.Linear(1000, vit_dim)
        
        model_name = "google/vit-base-patch16-224"
        config = LoraConfig(
                    task_type="CAUSAL_LM",
                    r=3,
                    lora_alpha=17,
                    target_modules=['query','key','value','dense'],
                    lora_dropout=0.01,
                    )
        modified_model = ModifiedViTForImageClassification.from_pretrained(model_name)
        self.lora_model = LoraModel(modified_model, config, "default")

        # Instance Normalization for ViT features
        self.instance_norm_vit = nn.InstanceNorm1d(vit_dim)

        # Freeze all ViT layers except the last two (LayerNorm and Classifier)
#         for name, param in self.vit.named_parameters():
# #             if 'heads' in name or 'norm' in name:
# #                 param.requires_grad = True  # Make last two layers trainable
# #             else:
# #                 param.requires_grad = False  # Freeze the rest of the ViT
#             param.requires_grad = True

        # Dropout for regularization
        self.dropout = nn.Dropout(p=dropout_p)

        # Classifier heads
        self.class_classifier = nn.Sequential(
            nn.Linear(vit_dim * 2, vit_dim),
            nn.ReLU(),
            nn.Dropout(p=dropout_p),  # Add dropout in the classifier
            nn.Linear(vit_dim, num_classes),
            nn.Softmax(dim=1)
        )

        self.domain_classifier = nn.Sequential(
            nn.Linear(vit_dim * 2, vit_dim),
            nn.ReLU(),
            nn.Dropout(p=dropout_p),  # Add dropout in the domain classifier
            nn.Linear(vit_dim, num_domains),
            nn.Softmax(dim=1)
        )

    def forward(self, x, alpha=1.0):
        # Extract features using pretrained ResNet
        resnet_features = self.resnet(x)
        resnet_features = resnet_features.view(resnet_features.size(0), -1)  # Flatten ResNet output
        resnet_features = self.resnet_to_vit_projection(resnet_features)  # Project ResNet features to match ViT dimensions
#         print(resnet_features.shape)

        # Instance normalization for ResNet features
        resnet_features = self.instance_norm_resnet(resnet_features)

        # Extract features using pretrained ViT
        vit_features = self.lora_model(x)

        # Instance normalization for ViT features
        vit_features = self.instance_norm_vit(vit_features)

        # Concatenate ResNet and ViT features
        combined_features = torch.cat((resnet_features, vit_features), dim=1)  # Concatenate along feature dimension

        # Apply dropout for regularization after concatenation
        combined_features = self.dropout(combined_features)

        # Classification output
        class_output = self.class_classifier(combined_features)

        # Apply GRL for domain classification
        reverse_features = GradientReversalLayer.apply(combined_features, alpha)
        domain_output = self.domain_classifier(reverse_features)

        return class_output, domain_output, combined_features



# Entropy minimization loss for domain classifier
def entropy_minimization_loss(predictions):
    epsilon = 1e-8  # To avoid log(0)
    entropy_loss = -torch.mean(torch.sum(predictions * torch.log(predictions + epsilon), dim=1))
    return entropy_loss

# Create the model
def create_custom_resnet_pretrained_vit_dann(num_classes=18, num_domains=4, vit_dim=768):
    return CustomResNetPretrainedViTDANN(num_classes=num_classes, num_domains=num_domains, vit_dim=vit_dim)

# Example usage
if __name__ == "__main__":
    model = create_custom_resnet_pretrained_vit_dann(num_classes=18, num_domains=4)
    
    # Example images (random for demonstration)
    img1 = torch.randn(1, 3, 224, 224)  # Example input image

    # Forward pass through the model
    class_output, domain_output, features = model(img1, alpha=1.0)
    
    # Simulated pairs for contrastive loss (random)
    features_a = features
    features_b = torch.randn_like(features)  # Example paired feature set
    contrastive_labels = torch.randint(0, 2, (features.size(0),))  # 0 or 1 labels for contrastive loss

    # Contrastive loss
    contrastive_criterion = ContrastiveLoss()
    contrastive_loss = contrastive_criterion(features_a, features_b, contrastive_labels)

    # Entropy minimization on domain output
    entropy_loss = entropy_minimization_loss(domain_output)

    print("Class Output shape:", class_output.shape)  # Should be [1, 18]
    print("Domain Output shape:", domain_output.shape)  # Should be [1, 4]
    print("Contrastive Loss:", contrastive_loss.item())
    print("Entropy Loss:", entropy_loss.item())
    print(features.shape)

Downloading: "https://download.pytorch.org/models/resnet152-f82ba261.pth" to /root/.cache/torch/hub/checkpoints/resnet152-f82ba261.pth
100%|██████████| 230M/230M [00:03<00:00, 78.6MB/s] 


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/instancenorm.py:88: UserWarning: input's size at dim=0 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
  warnings.warn(f"input's size at dim={feature_dim} does not match num_features. "


Class Output shape: torch.Size([1, 18])
Domain Output shape: torch.Size([1, 4])
Contrastive Loss: 0.5033111572265625
Entropy Loss: 1.1977946758270264
torch.Size([1, 1536])


In [4]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms,ops
from PIL import Image

# Custom dataset to handle both class and domain labels, excluding the test folder
class DomainClassDataset(Dataset):
    def __init__(self, root_dir, exclude_domain='test', transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.class_labels = []
        self.domain_labels = []
        self.domain_mapping = {}
        self.class_mapping = {}

        # Populate the image paths and labels, excluding the test domain
        for domain_idx, domain in enumerate(sorted(os.listdir(root_dir))):
            if domain == exclude_domain:  # Skip the test folder
                continue
            self.domain_mapping[domain] = domain_idx  # Assign a domain index
            domain_path = os.path.join(root_dir, domain)

            if os.path.isdir(domain_path):
                for class_name in sorted(os.listdir(domain_path)):
                    class_path = os.path.join(domain_path, class_name)
                    if os.path.isdir(class_path):
                        if class_name not in self.class_mapping:
                            self.class_mapping[class_name] = len(self.class_mapping)  # Assign a unique ID to each class
                        for img_name in os.listdir(class_path):
                            img_path = os.path.join(class_path, img_name)
                            if img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                                self.image_paths.append(img_path)
                                self.class_labels.append(self.class_mapping[class_name])
                                self.domain_labels.append(self.domain_mapping[domain])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        class_label = self.class_labels[idx]
        domain_label = self.domain_labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, class_label, domain_label

# Data transformation (normalization, etc.)
transform = transforms.Compose([
    transforms.Resize((224, 224), interpolation=transforms.InterpolationMode.BILINEAR),  # Resize with bilinear interpolation
    transforms.RandomHorizontalFlip(),          # Randomly flip horizontally
    transforms.ColorJitter(                     # Randomly change brightness, contrast, saturation, and hue
        brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2
    ),
    transforms.RandomGrayscale(p=0.1),          # Convert to grayscale with a probability
    transforms.RandomApply([                    # Randomly apply Gaussian blur
        transforms.GaussianBlur(kernel_size=(3, 3))], p=0.2),
    transforms.RandomRotation(degrees=15),      # Random rotation of up to 15 degrees
    transforms.ToTensor()  ,                    # Convert the image to a PyTorch tensor
    transforms.Normalize(                       # Normalize with ImageNet mean and std
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225] ) 
])


# Create the dataset for training and validation
dataset_root = '/kaggle/input/tidel-hack/tidel/dataset'
full_dataset = DomainClassDataset(root_dir=dataset_root, transform=transform)

# Train-validation split (80% train, 20% validation)
train_size = int(0.9 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# DataLoader for training and validation
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

# Load the test dataset, which contains only images (no class folders)
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = [os.path.join(root_dir, img_name) for img_name in os.listdir(root_dir) 
                            if img_name.lower().endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)

        return image, img_path  # Returning image path for reference

# Create the test dataset and DataLoader
test_transform = transforms.Compose([
    transforms.Resize((224, 224), interpolation=transforms.InterpolationMode.BILINEAR),  # Resize with bilinear interpolation
    transforms.ToTensor(),
    transforms.Normalize(                       # Normalize with ImageNet mean and std
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225] ) 
])
test_dataset = TestDataset(root_dir=os.path.join(dataset_root, 'test'), transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4)

# Debugging: Check the dataset information
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(val_dataset)}")
print(f"Number of classes: {len(full_dataset.class_mapping)}")
print(f"Number of domains: {len(full_dataset.domain_mapping)}")
print(f"Number of test samples: {len(test_dataset)}")


Number of training samples: 3939
Number of validation samples: 438
Number of classes: 18
Number of domains: 4
Number of test samples: 15620


In [5]:
def generate_adversarial_examples(model, images, labels, epsilon=0.01):
    images.requires_grad = True
    
    # Forward pass to get predictions
    outputs, _, _ = model(images)
    loss = torch.nn.CrossEntropyLoss()(outputs, labels)
    
    # Backward pass to calculate gradients
    model.zero_grad()
    loss.backward()
    
    # Generate adversarial images by adding a small perturbation
    adv_images = images + epsilon * images.grad.sign()
    adv_images = torch.clamp(adv_images, 0, 1)  # Ensure pixel values are valid
    
    return adv_images


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import os
from sklearn.metrics import f1_score

# Set CUDA_LAUNCH_BLOCKING=1 to enable synchronous CUDA calls for debugging
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Assuming ContrastiveLoss and EntropyMinimizationLoss are defined somewhere in your code
class ContrastiveLoss(nn.Module):
    def __init__(self, temperature=0.5):
        super(ContrastiveLoss, self).__init__()
        self.temperature = temperature

    def forward(self, z_i, z_j):
        # Assuming z_i and z_j are embeddings from positive/negative pairs
        batch_size = z_i.shape[0]
        z_i = nn.functional.normalize(z_i, dim=1)
        z_j = nn.functional.normalize(z_j, dim=1)
        
        # Positive similarity
        positive_sim = torch.exp(torch.sum(z_i * z_j, dim=-1) / self.temperature)
        
        # All similarities
        similarities = torch.exp(torch.mm(z_i, z_j.t()) / self.temperature)
        negative_sim = similarities.sum(dim=-1)
        
        loss = -torch.log(positive_sim / negative_sim)
        return loss.mean()

class EntropyMinimizationLoss(nn.Module):
    def __init__(self):
        super(EntropyMinimizationLoss, self).__init__()

    def forward(self, class_outputs):
        # Class_outputs is the predicted probability distributions (after softmax)
        p = torch.softmax(class_outputs, dim=1)
        log_p = torch.log(p + 1e-10)
        entropy = -torch.sum(p * log_p, dim=1)
        return entropy.mean()

def train_dann_with_adversarial(model, train_loader, val_loader, num_epochs=20, alpha=1.0, learning_rate=0.0001, epsilon=0.01):
    # Define loss functions
    classification_criterion = torch.nn.CrossEntropyLoss()
    domain_criterion = torch.nn.CrossEntropyLoss()
    contrastive_loss_fn = ContrastiveLoss(temperature=0.5)
    entropy_loss_fn = EntropyMinimizationLoss()

    # Optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    
    # Training Loop
    for epoch in range(num_epochs):
        model.train()
        running_class_loss = 0.0
        running_domain_loss = 0.0
        running_contrastive_loss = 0.0
        running_entropy_loss = 0.0
        running_corrects = 0
        total_samples = 0

        # To store true and predicted labels
        all_preds = []
        all_labels = []

        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 20)
        
        for images, class_labels, domain_labels in tqdm(train_loader, desc="Training"):
            images = images.cuda()  # Transfer to GPU if available
            class_labels = class_labels.cuda()
            domain_labels = domain_labels.cuda()
        
            optimizer.zero_grad()

            # Forward pass on clean data
            class_outputs, domain_outputs, embeddings = model(images, alpha=alpha)
            
            # Example for contrastive loss - This should be positive and negative pairs
            z_i, z_j = embeddings, embeddings  # This should be positive/negative pairs (adapt as per your data)
            contrastive_loss = contrastive_loss_fn(z_i, z_j)

            # Compute entropy minimization loss
            entropy_loss = entropy_loss_fn(class_outputs)

            # Compute classification and domain losses for clean data
            class_loss = classification_criterion(class_outputs, class_labels)#---------------
            # print(class_outputs.shape)
            # print(class_labels.shape)
            
            class_lab = torch.zeros_like(class_outputs).cuda()
            class_lab.scatter_(1, class_labels.unsqueeze(1), 1)

            domain_lab = torch.zeros_like(domain_outputs).cuda()
            domain_lab.scatter_(1, domain_labels.unsqueeze(1), 1)
            
#             class_loss = ops.sigmoid_focal_loss(class_outputs, class_lab,alpha = 0.25,gamma=1,reduction='sum')
            domain_loss = domain_criterion(domain_outputs, domain_labels)#--------------
            
            
#             domain_loss = ops.sigmoid_focal_loss(domain_outputs, domain_lab,alpha = 0.25,gamma=1,reduction='sum')

            # Adversarial training
            adv_images = generate_adversarial_examples(model, images, class_labels, epsilon)
            
            # Forward pass on adversarial examples
            adv_class_outputs, adv_domain_outputs, adv_embeddings = model(adv_images, alpha=alpha)
            
            # Compute classification and domain losses for adversarial examples
            adv_class_loss = classification_criterion(adv_class_outputs, class_labels) #---------------
#             adv_class_loss = ops.sigmoid_focal_loss(adv_class_outputs, class_lab,alpha = 0.25,gamma=1,reduction='sum')
            adv_domain_loss = domain_criterion(adv_domain_outputs, domain_labels) #-------------
#             adv_domain_loss = ops.sigmoid_focal_loss(adv_domain_outputs, domain_lab,alpha = 0.25,gamma=1,reduction='sum')
            
            
            # Total loss: combine clean and adversarial losses with weights
            total_class_loss = 0.5 * (class_loss + adv_class_loss)
            total_domain_loss = 0.5 * (domain_loss + adv_domain_loss)
            total_loss = total_class_loss + pow(3, total_domain_loss) + 0.4 * contrastive_loss + 0.2 * entropy_loss

            # Backward pass and optimization
            total_loss.backward()
            optimizer.step()

            # Track loss and accuracy
            running_class_loss += total_class_loss.item() * images.size(0)
            running_domain_loss += total_domain_loss.item() * images.size(0)
            running_contrastive_loss += contrastive_loss.item() * images.size(0)
            running_entropy_loss += entropy_loss.item() * images.size(0)
            
            # Predictions and true labels
            _, preds = torch.max(class_outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(class_labels.cpu().numpy())
            running_corrects += torch.sum(preds == class_labels.data)
            total_samples += images.size(0)

        epoch_class_loss = running_class_loss / total_samples
        epoch_domain_loss = running_domain_loss / total_samples
        epoch_entropy_loss = running_entropy_loss / total_samples
        epoch_acc = running_corrects.double() / total_samples

        # Calculate F1-score for the epoch
        epoch_f1_score = f1_score(all_labels, all_preds, average='weighted')

        print(f"Training - Classification Loss: {epoch_class_loss:.4f}, Domain Loss: {epoch_domain_loss:.4f}, Entropy Loss: {epoch_entropy_loss:.4f}, Accuracy: {epoch_acc:.4f}, F1-score: {epoch_f1_score:.4f}")

        # Validation phase
        val_class_loss, val_domain_loss, val_acc, val_f1 = evaluate_dann(model, val_loader, classification_criterion, domain_criterion, alpha)
        print(f"Validation - Classification Loss: {val_class_loss:.4f}, Domain Loss: {val_domain_loss:.4f}, Accuracy: {val_acc:.4f}, F1-score: {val_f1:.4f}")

        # Save model every 2 epochs
        if (epoch + 1) % 2 == 0:
            save_dir = "/kaggle/working/"
            os.makedirs(save_dir, exist_ok=True)
            torch.save(model.state_dict(), os.path.join(save_dir, f'model_epoch_{epoch + 1}.pth'))
            print(f'Model saved at epoch {epoch + 1}.')



# Evaluation function for validation
def evaluate_dann(model, val_loader, classification_criterion, domain_criterion, alpha):
    model.eval()
    running_class_loss = 0.0
    running_domain_loss = 0.0
    running_corrects = 0
    total_samples = 0

    # To store true and predicted labels
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, class_labels, domain_labels in tqdm(val_loader, desc="Validation"):
            images = images.cuda()  # Transfer to GPU if available
            class_labels = class_labels.cuda()
            domain_labels = domain_labels.cuda()

            # Forward pass
            class_outputs, domain_outputs, embeddings = model(images, alpha=alpha)

            # Calculate losses
            class_loss = classification_criterion(class_outputs, class_labels)
            domain_loss = domain_criterion(domain_outputs, domain_labels)

            # Track loss and accuracy
            running_class_loss += class_loss.item() * images.size(0)
            running_domain_loss += domain_loss.item() * images.size(0)
            _, preds = torch.max(class_outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(class_labels.cpu().numpy())
            running_corrects += torch.sum(preds == class_labels.data)
            total_samples += images.size(0)

    epoch_class_loss = running_class_loss / total_samples
    epoch_domain_loss = running_domain_loss / total_samples
    epoch_acc = running_corrects.double() / total_samples

    # Calculate F1-score for validation
    val_f1_score = f1_score(all_labels, all_preds, average='weighted')

    return epoch_class_loss, epoch_domain_loss, epoch_acc, val_f1_score

# Example usage
# if __name__ == "__main__":
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model = create_custom_resnet_dann(num_classes=18, num_domains=4).to(device)  # Move the model to GPU if available
#     model = create_pretrained_resnet_grl(num_classes=18, num_domains=4).to(device)
model = create_custom_resnet_pretrained_vit_dann(num_classes=18, num_domains=4).to(device)
print(device)
num_gpus = torch.cuda.device_count()
#     print(f"Number of GPUs: {num_gpus}")
model = nn.DataParallel(model)
#     model = torch.nn.DataParallel(model, device_ids=[0, 1]).to(device)
#     model = model.to(device)
# train_dann(model, train_loader, val_loader, num_epochs=20, alpha=1.0, learning_rate=0.00001)
train_dann_with_adversarial(model, train_loader, val_loader, num_epochs=20, alpha=1.0, learning_rate=0.00001)

cuda
Epoch 1/20
--------------------


Training:   0%|          | 0/124 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/instancenorm.py:88: UserWarning: input's size at dim=0 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
  warnings.warn(f"input's size at dim={feature_dim} does not match num_features. "
Training: 100%|██████████| 124/124 [06:01<00:00,  2.92s/it]


Training - Classification Loss: 2.8594, Domain Loss: 1.3669, Entropy Loss: 2.8897, Accuracy: 0.2110, F1-score: 0.1732


Validation:   0%|          | 0/14 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/instancenorm.py:88: UserWarning: input's size at dim=0 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
  warnings.warn(f"input's size at dim={feature_dim} does not match num_features. "
Validation: 100%|██████████| 14/14 [00:08<00:00,  1.72it/s]


Validation - Classification Loss: 2.8126, Domain Loss: 1.3226, Accuracy: 0.4064, F1-score: 0.3177
Epoch 2/20
--------------------


...

Validation - Classification Loss: 2.3126, Domain Loss: 1.3298, Accuracy: 0.7009, F1-score: 0.6246
Epoch 20/20
--------------------


Training:   0%|          | 0/124 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/instancenorm.py:88: UserWarning: input's size at dim=0 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
  warnings.warn(f"input's size at dim={feature_dim} does not match num_features. "
Training: 100%|██████████| 124/124 [06:04<00:00,  2.94s/it]


Training - Classification Loss: 2.3204, Domain Loss: 1.3693, Entropy Loss: 2.8543, Accuracy: 0.7200, F1-score: 0.6209


Validation:   0%|          | 0/14 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/instancenorm.py:88: UserWarning: input's size at dim=0 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
  warnings.warn(f"input's size at dim={feature_dim} does not match num_features. "
Validation: 100%|██████████| 14/14 [00:07<00:00,  1.82it/s]


Validation - Classification Loss: 2.5359, Domain Loss: 1.3339, Accuracy: 0.4315, F1-score: 0.4207
Model saved at epoch 20.


In [5]:
import pandas as pd
import torch
import os

results = []

# Set the model to evaluation mode
model.eval()

# Disable gradient calculation for inference
with torch.no_grad():
    # Add a tqdm progress bar to the loop
    for images, img_paths in tqdm(test_loader, desc="Processing Images", unit="image"):
        images = images.to(device)

        # Forward pass through the model
        class_output, domain_output, _ = model(images)

        # Get predicted class labels
        _, predicted_class = torch.max(class_output, 1)
        predicted_class = predicted_class.cpu().numpy()[0]  # Convert to numpy

        # Extract the image name without the extension
        image_name = os.path.splitext(os.path.basename(img_paths[0]))[0]

        # Store the result as (ID, TARGET)
        results.append({'ID': image_name, 'TARGET': predicted_class})

# Save the results to a CSV file
results_df = pd.DataFrame(results)
results_df.to_csv('predictions_2.csv', index=False)

print("Predictions saved to predictions_2.csv")

Processing Images:   0%|          | 0/15620 [00:00<?, ?image/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/instancenorm.py:88: UserWarning: input's size at dim=0 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
  warnings.warn(f"input's size at dim={feature_dim} does not match num_features. "
Processing Images: 100%|██████████| 15620/15620 [19:26<00:00, 13.39image/s]


Predictions saved to predictions_2.csv
